In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import time
base_url = 'https://rapzh.com'

# 读取歌手URL的CSV文件
with open('chinese_rappers.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    artists = list(reader)

# 打开用于写入歌曲数据的CSV文件
retry_list = []
with open('all_songs.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['rapper', 'song', 'url']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    # 遍历每个歌手
    for artist in tqdm(artists):
        url = artist['url'] + '/songs'

        while url:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            table = soup.find('table')
            if not table:
                print(f"未找到歌手 {artist['name']} 的歌曲列表")
                retry_list.append(artist)
                break

            for row in table.find_all('tr')[1:]:
                song_name = row.find_all('td')[0].text.strip()
                song_url = base_url + row.find_all('td')[0].find('a')['href']
                writer.writerow({'rapper': artist['name'], 'song': song_name, 'url': song_url})

            # 检查是否有下一页
            next_page = soup.find('a', class_='Pagination-next')
            if next_page:
                url = base_url + next_page['href']
            else:
                url = None
        time.sleep(5)

print('爬取完成,数据已保存到 all_songs.csv 文件中。')

In [17]:
import requests
from bs4 import BeautifulSoup
import csv

# 读取歌手URL的CSV文件
with open('chinese_rappers.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    artists = list(reader)

# 打开用于写入歌曲数据的CSV文件
with open('popular_songs.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['歌手', '歌名', '链接']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    # 遍历每个歌手
    for artist in tqdm(artists):
        url = artist['url']
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 找到所有的热门歌曲链接
        for song in soup.find_all('a', class_='song-title'):
            song_name = song.text.strip()
            song_url = 'https://rapzh.com' + song['href']
            writer.writerow({'歌手': artist['name'], '歌名': song_name, '链接': song_url})
        time.sleep(2)

print('爬取完成,数据已保存到 popular_songs.csv 文件中。')

100%|██████████| 3383/3383 [2:56:14<00:00,  3.13s/it]     

爬取完成,数据已保存到 popular_songs.csv 文件中。
